In [ ]:
# https://datasciencedojo.com/web-scraping-30-minutes/
# https://vimeo.com/209499033

In [1]:
from IPython.display import HTML

# Vimeo
HTML('<iframe src="https://player.vimeo.com/video/209499033?title=0&byline=0&portrait=0" width="700" height="394" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen>')

# Youtube
#HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/S_f2qV2_U00?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')


In [43]:
# following along with tutorial in the video above
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

my_url = 'https://www.newegg.com/Video-Cards-Video-Devices/Category/ID-38'

# opening up connection, grabbing the page
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

# html parsing
page_soup = soup(page_html, "html.parser")

# just checking if the data is in place and if I can read the header one
page_soup.h1
# or maybe the p tag
page_soup.p
page_soup.body.span

# use inspect in the browser to find the tag with the info on grapic cards
# first I find how to get the info on one card (the whole container), then I loop trough all the others
containers = page_soup.findAll('div', {'class':'item-container'})
# how many containers did we find?
'''len(containers)'''
# look at the first one
'''containers[0]'''

# to understand better - paste the output from container 1 into jsbeautifier.org
# follow the prettified html to the data I want to extract
'''container = containers[0]'''
# in this case the name of the board is in the image title within the a tag within div within the first div...
'''container.div.div.a.img['title']'''

# this is the last step - throw everything into a csv
filename = 'products.csv'
f = open(filename, 'w') # w for write
headers = 'brand, product_name, shipping\n'
f.write(headers)

# now use what we did to create a loop
for container in containers:
    brand = container.div.div.a.img['title']
    
    title_container = container.findAll('a', {'class' : 'item-title'})
    product_name = title_container[0].text
    
    shipping_container = container.findAll('li', {'class' : 'price-ship'})
    shipping = shipping_container[0].text.strip()
    
    print('brand: ' + brand)
    print('product_name: ' + product_name)
    print('shipping: ' + shipping)
    
    f.write(brand + ',' + product_name.replace(',', '|') + ',' + shipping + '\n') # replace commas in the text with pipes
    
f.close() # if we don't close the file we can't open it...

brand: GIGABYTE
product_name: GIGABYTE GeForce GTX 1050 Ti DirectX 12 GV-N105TWF2OC-4GD Video Card
shipping: $4.99 Shipping
brand: EVGA
product_name: EVGA GeForce GTX 1060 GAMING DirectX 12 06G-P4-6262-KR Video Card
shipping: $4.99 Shipping
brand: ASUS
product_name: ASUS ROG GeForce GTX 1070 STRIX-GTX1070-O8G-GAMING Video Card with RGB Lighting
shipping: $3.99 Shipping
brand: MSI
product_name: MSI GeForce GTX 1070 DirectX 12 GeForce GTX 1070 Quick Silver 8G OC Video Card
shipping: Free Shipping
brand: XFX
product_name: XFX Radeon RX 550 DirectX 12 RX-550P4TFG5 Video Card
shipping: $3.99 Shipping
brand: ZOTAC
product_name: ZOTAC GeForce GTX 1080 Ti AMP Edition 11GB GDDR5X 352-bit Gaming Graphics Card VR Ready 16+2 Power Phase Freeze Fan Stop IceStorm Cooling Spectra Lighting ZT-P10810D-10P
shipping: Free Shipping
brand: EVGA
product_name: EVGA GeForce GTX 1070 SC GAMING ACX 3.0 Black Edition, 08G-P4-5173-KR, 8GB GDDR5, LED, DX12 OSD Support (PXOC)
shipping: $4.99 Shipping
brand: GIGABYT

In [45]:
import pandas as pd
df_products = pd.read_csv('products.csv')
df_products.head()

,brand,product_name,shipping
0,GIGABYTE,GIGABYTE GeForce GTX 1050 Ti DirectX 12 GV-N10...,$4.99 Shipping
1,EVGA,EVGA GeForce GTX 1060 GAMING DirectX 12 06G-P4...,$4.99 Shipping
2,ASUS,ASUS ROG GeForce GTX 1070 STRIX-GTX1070-O8G-GA...,$3.99 Shipping
3,MSI,MSI GeForce GTX 1070 DirectX 12 GeForce GTX 10...,Free Shipping
4,XFX,XFX Radeon RX 550 DirectX 12 RX-550P4TFG5 Vide...,$3.99 Shipping


In [1]:
from bs4 import BeautifulSoup as soup  # HTML data structure
from urllib.request import urlopen as uReq  # Web client

# URl to web scrap from.
# in this example we web scrap graphics cards from Newegg.com
page_url = "http://www.newegg.com/Product/ProductList.aspx?Submit=ENE&N=-1&IsNodeId=1&Description=GTX&bop=And&Page=1&PageSize=36&order=BESTMATCH"

# opens the connection and downloads html page from url
uClient = uReq(page_url)

# parses html into a soup data structure to traverse html
# as if it were a json data type.
page_soup = soup(uClient.read(), "html.parser")
uClient.close()

# finds each product from the store page
containers = page_soup.findAll("div", {"class": "item-container"})

# name the output file to write to local disk
out_filename = "graphics_cards.csv"
# header of csv file to be written
headers = "brand,product_name,shipping\n"

# opens file, and writes headers
f = open(out_filename, "w")
f.write(headers)

# loops over each product and grabs attributes about
# each product
for container in containers:
    # Finds all link tags "a" from within the first div.
    make_rating_sp = container.div.select("a")

    # Grabs the title from the image title attribute
    # Then does proper casing using .title()
    brand = make_rating_sp[0].img["title"].title()

    # Grabs the text within the second "(a)" tag from within
    # the list of queries.
    product_name = container.div.select("a")[2].text

    # Grabs the product shipping information by searching
    # all lists with the class "price-ship".
    # Then cleans the text of white space with strip()
    # Cleans the strip of "Shipping $" if it exists to just get number
    shipping = container.findAll("li", {"class": "price-ship"})[0].text.strip().replace("$", "").replace(" Shipping", "")

    # prints the dataset to console
    print("brand: " + brand + "\n")
    print("product_name: " + product_name + "\n")
    print("shipping: " + shipping + "\n")

    # writes the dataset to file
    f.write(brand + ", " + product_name.replace(",", "|") + ", " + shipping + "\n")

f.close()  # Close the file